In [1]:
import time
import adodbapi
import pandas as pd
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
client_manuf = ["Edgewell Personal Care"]
client_brands = ["Schick","Equate","Cremo"]
decimals = 2
sign = "Before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

prodORitem="SKU"
categories =["Manual Shave Men"]
sectors = ["System","Disposables"]
segments = ["Razors","Refills","Disposables"]
subsegments= []
subcategories= []

customareas=''
national = True
areas = ['NATIONAL','RETAILER']

regions_RET  =["Bj's And Sam's","Walmart"]
channels_RET = []
market_RET = []

regions_REG = []
channels_REG = []
market_REG = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = [] 
 
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending March  2025"
end_date = "2025-04-01"
years = {2023,2024,2025}

ManufOrTopC ="Top Companies"
BrandOrTopB = "Brand"

past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
past_3_months = pd.date_range(end=end_date, periods=3, freq='ME').strftime('%b-%y').tolist()
past_36_months = pd.date_range(end=end_date, periods=36, freq='ME').strftime('%b-%y').tolist()

National=["NATIONAL"]if national else[]

entity_hierarchy = [
    ("NATIONAL","Area",National),
    ("RETAILER","Region", regions_RET),
    ("RETAILER","Channel", channels_RET),
    ("RETAILER","Market", market_RET),
    
    ("CHANNEL","Region", regions_CHAN),
    ("CHANNEL","Channel", channels_CHAN),
    ("CHANNEL","Market", market_CHAN),
    
    ("REGION","Region", regions_REG),
    ("REGION","Channel", channels_REG),
    ("REGION","Market", market_REG),
    
    (f"{customareas}","Region", regions_CUST),
    (f"{customareas}","Channel", channels_CUST),
    (f"{customareas}","Market", market_CUST)
]
entity_hierarchy = [
    item for item in entity_hierarchy
    if len(item) == 3 and item[2]  # item[2] is the entity list
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
 
]
direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}
server = "powerbi://api.powerbi.com/v1.0/myorg/Edgewell"
dataset_name = "Edgewell US Male Dataset"
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"
p3m_dax = "{" + ", ".join(f'"{date}"' for date in past_3_months) + "}"
p36m_dax = "{" + ", ".join(f'"{date}"' for date in past_36_months) + "}"

print(p12m_dax)
path=os.path.join(os.getcwd(),"Pricing Datasets NewEX")

conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=900;"
print(conn_str)

{"Apr-24", "May-24", "Jun-24", "Jul-24", "Aug-24", "Sep-24", "Oct-24", "Nov-24", "Dec-24", "Jan-25", "Feb-25", "Mar-25"}
Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Edgewell;Initial Catalog=Edgewell US Male Dataset;Timeout=900;


## Price Positioning

In [3]:
def execute_dax_query(BrandorManuf,entity_name,entity_type, area,market, hierby):
    outputdic = {}
    key =  f"{entity_type} | {entity_name}"


    columns = [
        "Relative Price",'Av Price/Unit','Value Sales','Value Share','Value Share DYA','Av Price/KG','IYA Price/KG'
    ]
    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Main query 
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{BrandorManuf}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Grand total query (not grouped by BrandOrTopB, just Category)
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            grand_tot[df.columns[0]] = 'Grand Total'

            # Reorder columns if necessary
            grand_tot = grand_tot[df.columns]

            # Concatenate the two
            df = pd.concat([df, grand_tot], ignore_index=True)
            outputdic[key] = df  

            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic



def process_dax_queries(BrandorManuf,entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{value} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query,BrandorManuf, entity,value,area,market, hierby)
                            futures[future] = key
       
        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        if BrandorManuf==f'{BrandOrTopB}':
            filename =  f"price_positioning_brands.pkl"
        else:
            filename =  f"price_positioning_manuf.pkl"
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")

process_dax_queries(f'{BrandOrTopB}',entity_hierarchy,hierarchy_levels) 
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy,hierarchy_levels) 

Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
All DataFrames saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_positioning_brands.pkl.
Query executed successfully for Bj's And Sam's.
Query executed succe

## Sector/Segment/SubSegment Leaderships (SINGLE)

In [4]:
def execute_dax_query(entity_name, area,market, hierby, client, total=True):
    outputdic = {}
    key = f"{client} | {entity_name}" if client else f"{entity_name}"

    columns = [
        "Value Share", "Value Sales", "Av Price/KG", "WoB %", "Gross Margin %"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Prepare client brand filter conditionally
    client_filter = ""
    if not total and client:
        client_filter = f'''
            FILTER(
                Products,
                Products[Top Brands] = "{client}"
            ),
        '''

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        grand_tot[df.columns[0]] = 'Grand Total'
        grand_tot = grand_tot[df.columns]
        df = pd.concat([df, grand_tot], ignore_index=True)
        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None, total=False):
    with ThreadPoolExecutor(max_workers=8) as executor:
        
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                if client_brands: 
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            for client in client_brands:
                                key = f"{client} | {entity}"
                                ordered_keys.append(key)
                                future = executor.submit(
                                    execute_dax_query, entity, area,market, hierby, client, total
                                )
                                futures[future] = key
                else:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{entity}"
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query, entity, area,market, hierby, '', total
                            )
                            futures[future] = key

            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            if client_brands:
                filename = f"{hierby}_leadership.pkl"
            else:
                filename = f"{hierby}_total_leadership.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

# Execute
if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands)

process_dax_queries(entity_hierarchy, hierarchy_levels, total=True)


Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
All DataFrames for Category saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\Category_leadership.pkl.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
All DataFrames for Sector saved to c:\Users\AleaaSALAH\Docum

## Sector/Segment Leaderships

In [5]:
def execute_dax_query(entity_name, area,market, hierby,direct_parent, client,manuf, total=False):
    outputdic = {}
    key = f"{client} | {entity_name}" if client else (f"{manuf} | {entity_name}" if manuf else f"{entity_name}")

    columns = [
        "Value Share", "Value Sales", "Av Price/KG", "WoB %", "Gross Margin %"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Prepare client brand filter conditionally
    client_filter = ""
    if not total and client:
        client_filter = f'''
            FILTER(
                Products,
                Products[Top Brands] = "{client}"
            ),
        '''    
    manuf_filter=''
    if not total and manuf:
        manuf_filter = f'''
            FILTER(
                Products,
                Products[Top Companies] = "{manuf}"
            ),
        '''

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}],
                    Products[{hierby}]
                    
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}]                   
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
        # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"

        if maintotal_df.empty:
            outputdic[key] = maintotal_df
            return outputdic
        if not maintotal_df.empty:
            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
    
        if not grand_tot.empty:
            # Create a dict for the first two columns
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan  # or pd.NA

            # Ensure all required columns exist, fill missing ones with NaN
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan

            # Reorder columns exactly as in df
            grand_tot = grand_tot[df.columns]
            df = pd.concat([df_with_totals, grand_tot], ignore_index=True)

        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, client_brands=None,client_manuf=None, total=False):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category" or direct_parent[hierby]=="Category":
                    continue
            if isinstance(hier_values, list):
                if client_brands and not total: 
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            for client in client_brands:
                                key = f"{client} | {entity}"
                                ordered_keys.append(key)
                                future = executor.submit(
                                    execute_dax_query, entity, area,market, hierby,direct_parent, client,'', ''
                                )
                                futures[future] = key
                if client_manuf and not total:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            for manuf in client_manuf:
                                key = f"{manuf} | {entity}"
                                ordered_keys.append(key)
                                future = executor.submit(
                                    execute_dax_query, entity, area,market, hierby,direct_parent, '',manuf,''
                                )
                                futures[future] = key               
                elif total==True:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{entity}"
                            print("SALMA YA SALAMA")
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query, entity, area,market, hierby, direct_parent,'','', total
                            )
                            futures[future] = key

            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            if client_brands:
                filename = f"brand_{hierby}.pkl"
            elif client_manuf:
                filename = f"manuf_{hierby}.pkl"
            else:
                filename = f"total_{hierby}.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, client_brands=client_brands)
if client_manuf:
    process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, client_manuf=client_manuf)   
process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, total=True)


Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
All DataFrames for Segment saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\brand_Segment.pkl.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
All DataFrames for SubSegment saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\brand_SubSegment.pkl.
Query exec

## Shelf & Avg Price / KG By Brand

In [6]:
def execute_dax_query(manuforbrand,entity_name,entity_type, area,market, hierby, incall=True):
    outputdic = {}
    key =  f"{entity_type} | {entity_name}"
    if manuforbrand==f'{BrandOrTopB}':
        columns = [
            "Relative Price", "Av Price/Unit", "Value Sales", "IYA Price/KG", "Base Price/KG","Av Price/KG","Value Share","WoB %","Value Share DYA"
        ]
    else:
        columns = ["Base Price/KG","Av Price/KG","Value Share","WoB %","Value Share DYA"]
 
 
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
 
    # Prepare client brand filter conditionally
    scope_filter = ""
    if incall:
        scope_filter = f'''
            FILTER('Scope', 'Scope'[Scope] = "Category"),
            Products[{direct_parent[hierby]}] = "{entity_type}"
            
        '''
    else:
        scope_filter = f'''
            FILTER('Scope', 'Scope'[Scope] = "{hierby}"),
            Products[{direct_parent[hierby]}] = "{entity_type}"
        '''
 
    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}],
                    Products[{manuforbrand}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            {scope_filter}
        )
    """
    parenttotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}]
 
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            {scope_filter}
        )
    """
   
    # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            {scope_filter}
 
        )
    """
 
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
 
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
 
        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
 
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
       
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
 
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
 
        if maintotal_df.empty:
            outputdic[key] = maintotal_df
            return outputdic
        if not maintotal_df.empty:
            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
 
        if not grand_tot.empty:
            # Ensure 'Grand Total' label is added to the first column of grand_tot
            grand_tot[df_with_totals.columns[0]] = 'Grand Total'
 
            # Keep only columns that exist in df_with_totals
            common_columns = [col for col in df_with_totals.columns if col in grand_tot.columns]
            grand_tot = grand_tot[common_columns]
 
            # Align grand_tot with df_with_totals in case column order matters
            grand_tot = grand_tot.reindex(columns=df_with_totals.columns)
 
        df = pd.concat([df_with_totals, grand_tot], ignore_index=True)
        outputdic[key] = df
 
 
        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")
 
    return outputdic
 
 
def process_dax_queries(manuforbrand,entity_hierarchy, hierarchy_levels, incall=False):
    with ThreadPoolExecutor(max_workers=8) as executor:
        hierarchy_dict = dict(hierarchy_levels)
 
        for hierby, _ in hierarchy_levels:
            if hierby == "Category":
                continue
                   
            dfs_results = {}
            futures = {}
            ordered_keys = []
            # Get the parent level name (e.g., 'Segment')
            parent_level = direct_parent[hierby]
            parent_values = hierarchy_dict.get(parent_level, [])
            if isinstance(parent_values, list):
                for value in parent_values:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{value} | {entity}"                          
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query,manuforbrand, entity, value, area,market, hierby,incall
                            )
                            futures[future] = key
            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)
 
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            if manuforbrand==f'{BrandOrTopB}':
                if incall==False:
                    filename = f"shelf_{hierby}_top_brands.pkl"
                else:
                    filename = f"shelf_{hierby}_all_brands.pkl"
            else:
                if incall==False:
                    filename = f"shelf_{hierby}_top_manuf.pkl"
                else:
                    filename = f"shelf_{hierby}_all_manuf.pkl"
 
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
           
            print(f"All DataFrames for {hierby} saved to {output_file}.")
 
 
 
# Execute
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels,incall=False)
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels, incall=True)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,incall=False)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels, incall=True)

Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
All DataFrames for Sector saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\shelf_Sector_top_brands.pkl.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
All DataFrames for Segment saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\shelf_Segment_top_brands.pkl.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successful

## Price Point Distribution By Product (Item)


In [7]:
def execute_dax_query(entity_name, entity_type, area,market, hierby, client, manuf,p12m=True):
    outputdic = {}

    key = f"{client} | {entity_name} | {entity_type}" if client else f"{manuf} | {entity_name} | {entity_type}" 
    if p12m :
        timeper=p12m_dax
    else:
        timeper=p3m_dax    
    columns = ["Base Price/Unit", "Base Price/KG", "Value Sales", "Gross Margin %"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    client_filter=''
    if client:
        client_filter = f'''
                FILTER(
                    Products,
                    Products[{BrandOrTopB}] = "{client}" &&
                    Products[{ManufOrTopC}] = "{manuf}"
                ),
            '''
   
    manuf_filter = f'FILTER(Products, Products[Top Companies] = "{manuf}"),' if manuf else ""

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}],
                    Products[{prodORitem}],
                    Products[Total Size]
                ),
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """

    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{hierby}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    itemtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products,Products[{prodORitem}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

            with conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
          
            with conn.cursor() as cursor:
                cursor.execute(itemtotal_dax_query)
                itemtotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])


            with conn.cursor() as cursor:
                cursor.execute(grandtotal_query)
                grand_tot = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

        for dataframe in [df,maintotal_df,itemtotal_df, grand_tot]:
            dataframe.columns = dataframe.columns.str.replace(r'.*\[|\]', '', regex=True)
            dataframe.dropna(how='all', inplace=True)

        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, maintotal_df], ignore_index=True)
      
        if not itemtotal_df.empty:
            itemtotal_df.iloc[:, 0] = itemtotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, itemtotal_df], ignore_index=True)
            
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)

        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None, client_manuf=None,p12m=True):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue

            hierarchy_dict = dict(hierarchy_levels)
            parent_level = direct_parent.get(hierby)
            parent_values = hierarchy_dict.get(parent_level, [])

            if not parent_values:
                continue

            futures = {}
            ordered_keys = []

            # Determine prodORitem per level

            for value in parent_values:
                for area,market, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        if client_brands:
                            for client in client_brands:
                                for manuf in client_manuf:
                                    key = f"{client} | {entity} | {value}"
                                    ordered_keys.append(key)
                                    futures[executor.submit(
                                        execute_dax_query, entity, value, area,market, hierby, client, manuf,p12m
                                    )] = key
                        elif client_manuf:
                            for manuf in client_manuf:
                                key = f"{manuf} | {entity} | {value}"
                                ordered_keys.append(key)
                                futures[executor.submit(
                                    execute_dax_query, entity, value, area,market, hierby, None, manuf, p12m
                                )] = key

            dfs_results = {}
            for future in as_completed(futures):
                result = future.result()
                dfs_results.update(result)

            dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}
            if p12m:
                 filename = f"price_distribution_{hierby}_{'p12m' if client_brands else 'manuf_p12m'}.pkl"
            else:
                 filename = f"price_distribution_{hierby}_{'p3m' if client_brands else 'manuf_p3m'}.pkl"
                     
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            print(f"All DataFrames for {hierby} saved to {output_file}.")


# Example Invocation
if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands,client_manuf=client_manuf,p12m=True)
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands,client_manuf=client_manuf,p12m=False)

process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf,p12m=True)
process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf,p12m=False)




Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
All DataFrames for Sector saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_Sector_p12m.pkl.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query exe

C:\Users\AleaaSALAH\AppData\Local\Temp\ipykernel_12928\4291102068.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)
C:\Users\AleaaSALAH\AppData\Local\Temp\ipykernel_12928\4291102068.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)


Query executed successfully for Walmart.
Query executed successfully for Walmart.


C:\Users\AleaaSALAH\AppData\Local\Temp\ipykernel_12928\4291102068.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)


Query executed successfully for Walmart.
All DataFrames for Segment saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_Segment_p12m.pkl.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query e

...

Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.


C:\Users\AleaaSALAH\AppData\Local\Temp\ipykernel_12928\4291102068.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)


Query executed successfully for Walmart.
Query executed successfully for Walmart.


C:\Users\AleaaSALAH\AppData\Local\Temp\ipykernel_12928\4291102068.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)


Query executed successfully for Walmart.
All DataFrames for SubCategory saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_SubCategory_p3m.pkl.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
All DataFrames for Sector saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_Sector_manuf_p12m.pkl.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
All DataFrames for Segment saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_Segment_manuf_p12m.pkl.
Query executed successfully for Bj's And Sam's.
Query 

## Price Point Distribution By Product Scraped(Item)

In [ ]:
def execute_dax_query(entity_name,entity_type, area,market, hierby, client, manuf):
    outputdic={}
    key = f"{client} | {entity_name} | {entity_type}" if client else f"{manuf} | {entity_name} | {entity_type}" 
    
    columns = [
        "Scraped Av. Price/Unit", "Scraped Av. Price/KG"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    client_filter = ""
    client_filter=""
    if client:
        client_filter = f'''
                Products[Top Brands] = "{client}"&&
                
        '''
    if manuf:
        manuf_filter = f'''
                Products[Top Companies] = "{manuf}"
        '''
        

    # Main query
    dax_query = f"""
    EVALUATE
    SUMMARIZECOLUMNS(   
        Calendar[End of Week],
        Products[{hierby}],
        Products[{prodORitem}],
        Products[Ean],
        Products[Total Size],
        FILTER(
            Products,
                {client_filter}
                {manuf_filter}
                && Products[Category] = "Manual Shave Men"
                && Products[{direct_parent[hierby]}] = "{entity_type}"
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({{ "{entity_name}" }}, Market[{market}]),
        FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}"),
        "Scraped Av. Price/Unit", [Scraped Av. Price/Unit],
        "Scraped Av. Price/KG", [Scraped Av. Price/KG]
    )
    """
    parenttotal_dax_query = f"""
    EVALUATE
    SUMMARIZECOLUMNS(   
        Calendar[End of Week],
        Products[{hierby}],

        FILTER(
            Products,
                {client_filter}
                {manuf_filter}
                && Products[Category] = "Manual Shave Men"
                && Products[{direct_parent[hierby]}] = "{entity_type}"
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({{ "{entity_name}" }}, Market[{market}]),
        FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}"),
        "Scraped Av. Price/Unit", [Scraped Av. Price/Unit],
        "Scraped Av. Price/KG", [Scraped Av. Price/KG]
    )
    """


    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
            
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)

        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 1] = maintotal_df.iloc[:, 1].astype(str) + " Total"

        if not maintotal_df.empty:
            df = pd.concat([df,maintotal_df], ignore_index=True)

        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None,client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue

            hierarchy_dict = dict(hierarchy_levels)
            parent_level = direct_parent.get(hierby)
            parent_values = hierarchy_dict.get(parent_level, [])

            if not parent_values:
                continue

            futures = {}
            ordered_keys = []

            # Determine prodORitem per level

            for value in parent_values:
                for area,market, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        if client_brands:
                            for client in client_brands:
                                for manuf in client_manuf:
                                    key = f"{client} | {entity} | {value}"
                                    ordered_keys.append(key)
                                    futures[executor.submit(
                                        execute_dax_query, entity, value, area,market, hierby, client, manuf
                                    )] = key
                        elif client_manuf:
                            for manuf in client_manuf:
                                key = f"{manuf} | {entity} | {value}"
                                ordered_keys.append(key)
                                futures[executor.submit(
                                    execute_dax_query, entity, value, area,market, hierby, None, manuf
                                )] = key

            dfs_results = {}
            for future in as_completed(futures):
                result = future.result()
                dfs_results.update(result)

            dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}

            if client_brands:
                filename = f"price_distribution_scraped_{hierby}.pkl"
            else:
                filename = f"price_distribution_scraped_{hierby}_manuf.pkl"


            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands,client_manuf=client_manuf)
if client_manuf:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf)   


Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
     End of Week        Sector  \
0     2024-08-04        System   
1     2024-08-04   Disposables   
2     2024-08-04        System   
3     2024-08-04   Disposables   
4     2024-08-04   Disposables   
...          ...           ...   
1066  2025-08-03  System Total   
1067  2025-08-10  System Total   
1068  2025-08-17  System Total   
1069  2025-08-24  System Total   
1070  2025-08-31  System Total   

                                                    SKU         Ean  \
0        HYDRO 5 SENSITIVE 5 BLADE REFILL SENSITIVE 6CT  4105801480   
1     SCHICK XTREME 3 SENSITIVE BLACK 3 BLADE DISPOS...  4105802095   
2       HYDRO 5 SENSITIVE 5 BLADE REFILL SENSITIVE 12CT  4105801598

In [ ]:
def execute_dax_query(entity_name,entity_type, area,market, hierby, client, manuf):
    outputdic={}
    key = f"{client} | {entity_name} | {entity_type}" if client else f"{manuf} | {entity_name} | {entity_type}" 
    
    columns = [
        "Value Sales"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    client_filter = ""
    client_filter=""
    if client:
        client_filter = f'''
                Products[Top Brands] = "{client}"&&
                
        '''
    if manuf:
        manuf_filter = f'''
                Products[Top Companies] = "{manuf}"
        '''
        

    # Main query
    dax_query = f"""
    EVALUATE
    SUMMARIZECOLUMNS(  
        Products[Ean],
   
        FILTER(
            Products,
                {client_filter}
                {manuf_filter}
                && Products[Category] = "Manual Shave Men"
                && Products[{direct_parent[hierby]}] = "{entity_type}"
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({{ "{entity_name}" }}, Market[{market}]),
        FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}"),
        "Value Sales", [Value Sales]
    )
    """
    


    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
 
        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None,client_manuf=None):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue

            hierarchy_dict = dict(hierarchy_levels)
            parent_level = direct_parent.get(hierby)
            parent_values = hierarchy_dict.get(parent_level, [])

            if not parent_values:
                continue

            futures = {}
            ordered_keys = []

            # Determine prodORitem per level

            for value in parent_values:
                for area,market, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        if client_brands:
                            for client in client_brands:
                                for manuf in client_manuf:
                                    key = f"{client} | {entity} | {value}"
                                    ordered_keys.append(key)
                                    futures[executor.submit(
                                        execute_dax_query, entity, value, area,market, hierby, client, manuf
                                    )] = key
                        elif client_manuf:
                            for manuf in client_manuf:
                                key = f"{manuf} | {entity} | {value}"
                                ordered_keys.append(key)
                                futures[executor.submit(
                                    execute_dax_query, entity, value, area,market, hierby, None, manuf
                                )] = key

            dfs_results = {}
            for future in as_completed(futures):
                result = future.result()
                dfs_results.update(result)

            dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}

            if client_brands:
                filename = f"price_distribution_scraped_{hierby}_Ean.pkl"
            else:
                filename = f"price_distribution_scraped_{hierby}_Ean_manuf.pkl"


            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands,client_manuf=client_manuf)
if client_manuf:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf)   


Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
All DataFrames for Sector saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_scraped_Sector_Ean.pkl.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Q

## Price Point Comparison by Product (scraped)

In [ ]:
def execute_dax_query(BrandORManuf,entity_name,entity_type, area,market, hierby):
    outputdic={}
    key = f"{entity_name} | {entity_type}" 
    

    # Main query
    dax_query = f"""
    EVALUATE
    SUMMARIZECOLUMNS(   
        Calendar[End of Week],
        Products[{BrandORManuf}],
        Products[{prodORitem}],
        Products[Ean],
        Products[Total Size],
        FILTER(
            Products,
                Products[Category] = "Manual Shave Men"
                && Products[{hierby}] = "{entity_type}"
        ),
        FILTER('Market', 'Market'[Area] = "{area}"),
        TREATAS({{ "{entity_name}" }}, Market[{market}]),
        FILTER('Scope', 'Scope'[Scope] = "{hierby}"),
        "Scraped Av. Price/Unit", [Scraped Av. Price/Unit],
        "Scraped Av. Price/KG", [Scraped Av. Price/KG]
    )
    """


    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)

        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(BrandORManuf,entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{entity} | {value}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query,BrandORManuf, entity,value,area,market, hierby)
                            futures[future] = key
       
        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        if BrandORManuf==f"{BrandOrTopB}":         
            filename = f"price_point_by_brands_items_scraped.pkl"
        else:
            filename = f"price_point_by_manuf_items_scraped.pkl"
            
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")

process_dax_queries(f"{BrandOrTopB}",entity_hierarchy, hierarchy_levels)
process_dax_queries(f"{ManufOrTopC}",entity_hierarchy, hierarchy_levels)

 


Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
All DataFrames saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_point_by_brands_items_scraped.pkl.
Query executed successfully for Bj's And Sam's.
Query exe

## Price Point Distribution By Brand

In [11]:
def execute_dax_query(entity_name, entity_type, area,market, hierby):
    outputdic = {}

    key = f"{entity_name} | {entity_type}" 


    columns = "Av Price/Unit", "Value Share"
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{BrandOrTopB}],
                    Products[Pack Size]
                ),
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{BrandOrTopB}]),
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
        
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            Products[{hierby}] = "{entity_type}",
            
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")

        )
    """

    try:
        # 1. Main query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            data_columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

        # 2. Parent total query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()

        # 3. Grand total query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        # Now build DataFrames safely
        df = pd.DataFrame(data, columns=data_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        

        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        



        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df,maintotal_df], ignore_index=True)
  
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)

        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{entity} | {value}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query, entity,value,area,market, hierby)
                            futures[future] = key
       
        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        filename = f"price_distribution_by_brands_category.pkl"
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")

process_dax_queries(entity_hierarchy,hierarchy_levels) 



Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
All DataFrames saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_by_brands_category.pkl.


## By brand For Sec,Seg,SubSeg... (Slide 7)


In [12]:
def execute_dax_query(entity_name, entity_type, area,market, hierby):
    outputdic = {}
 
    key = f"{entity_name} | {entity_type}"
    columns = "Av Price/Unit", "Value Share"
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
             
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{BrandOrTopB}],
                    Products[{hierby}],
                    Products[Pack Size]
                ),
                
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """
 
    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{BrandOrTopB}]),
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    itemtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products,Products[{BrandOrTopB}],
                Products[{hierby}]),
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
 
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}])
        )
    """
 
    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
 
            with conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
         
            with conn.cursor() as cursor:
                cursor.execute(itemtotal_dax_query)
                itemtotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
 
 
            with conn.cursor() as cursor:
                cursor.execute(grandtotal_query)
                grand_tot = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
 
        for dataframe in [df,maintotal_df,itemtotal_df, grand_tot]:
            dataframe.columns = dataframe.columns.str.replace(r'.*\[|\]', '', regex=True)
            dataframe.dropna(how='all', inplace=True)
       
            cols_to_check = ["Av Price/Unit", "Value Share"]
            dataframe.drop(
                index=dataframe[
                    (dataframe[cols_to_check].abs() < 1e-6).all(axis=1)
                ].index,
                inplace=True
            )    
           
 
        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, maintotal_df], ignore_index=True)
     
        if not itemtotal_df.empty:
            itemtotal_df.iloc[:, 1] = itemtotal_df.iloc[:, 1].astype(str) + " Total"
            df = pd.concat([df, itemtotal_df], ignore_index=True)
           
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)
 
        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")
 
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")
 
    return outputdic
 
 
def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue
 
            hierarchy_dict = dict(hierarchy_levels)
            parent_level = direct_parent.get(hierby)
            parent_values = hierarchy_dict.get(parent_level, [])
 
            if not parent_values:
                continue
 
            futures = {}
            ordered_keys = []
 
            # Determine prodORitem per level
 
            for value in parent_values:
                for area,market, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        for manuf in client_manuf:
                            key = f"{entity} | {value}"
                            ordered_keys.append(key)
                            futures[executor.submit(
                                execute_dax_query, entity, value, area,market, hierby
                            )] = key
 
            dfs_results = {}
            for future in as_completed(futures):
                result = future.result()
                dfs_results.update(result)
 
            dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}
           
            filename = f"price_distribution_by_brands_{hierby}"
                     
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            print(f"All DataFrames for {hierby} saved to {output_file}.")
 
 
process_dax_queries(entity_hierarchy, hierarchy_levels)
 
 
 

Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
All DataFrames for Sector saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_by_brands_Sector.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
All DataFrames for Segment saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_by_brands_Segment.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed 

## Price Point Comparison by Product

In [13]:
def execute_dax_query(ManuforBrand,entity_name, entity_type, area,market, hierby, p12m=True):
    outputdic = {}

    key = f"{entity_name} | {entity_type}"

    if p12m:
        timeper = p12m_dax
        columns = ["Value Share", "Gross Margin %"]
        row_fields = [
            f"Products[{ManuforBrand}]",
            f"Products[{prodORitem}]"
        ]
    else:
        timeper = p3m_dax
        columns = ["Base Price/Unit", "Base Price/KG", "Value Sales", "Value Share"]
        row_fields = [
            f"Products[{ManuforBrand}]",
            f"Products[{prodORitem}]",
            "Products[Total Size]"
        ]

    summarize_fields = ",\n                        ".join(row_fields)
    column_exprs = ",\n                ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    {summarize_fields}
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS({timeper}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{ManuforBrand}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """
    itemtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products,Products[{prodORitem}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE( 
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            TREATAS({timeper}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
            
        )
    """

    try:
        
        
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

            with conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
          
            with conn.cursor() as cursor:
                cursor.execute(itemtotal_dax_query)
                itemtotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])


            with conn.cursor() as cursor:
                cursor.execute(grandtotal_query)
                grand_tot = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
          
            numeric_columns = columns
            for i, dataframe in enumerate([df, maintotal_df, itemtotal_df, grand_tot]):
                dataframe.columns = dataframe.columns.str.replace(r'.*\[|\]', '', regex=True)
                dataframe.dropna(how='all', inplace=True)

                if not dataframe.empty:
                    # Force conversion to numeric in case some values are strings
                    dataframe[numeric_columns] = dataframe[numeric_columns].apply(pd.to_numeric, errors='coerce')

                    # Now drop rows where all numeric values are 0.0
                    mask_all_zero = (dataframe[numeric_columns].fillna(0.0) == 0.0).all(axis=1)
                    dataframe = dataframe[~mask_all_zero]

                if i == 0:
                    df = dataframe
                elif i == 1:
                    maintotal_df = dataframe
                elif i == 2:
                    itemtotal_df = dataframe
                else:
                    grand_tot = dataframe



        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, maintotal_df], ignore_index=True)
            
        if not p12m:
            if not itemtotal_df.empty:
                itemtotal_df.iloc[:, 0] = itemtotal_df.iloc[:, 0].astype(str) + " Total"
                df = pd.concat([df, itemtotal_df], ignore_index=True)
            
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)

        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic


def process_dax_queries(ManuforBrand,entity_hierarchy, hierarchy_levels,p12m):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{entity} | {value}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query,ManuforBrand, entity,value,area,market, hierby,p12m)
                            futures[future] = key

        dfs_results = {}
        for future in as_completed(futures):
            result = future.result()
            dfs_results.update(result)

        dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}
        if ManuforBrand ==f'{BrandOrTopB}' :
            if p12m:
                    filename = f"price_point_by_brands_items_P12M.pkl"
            else:
                    filename = f"price_point_by_brands_items_P3M.pkl"
        else:
            if p12m:
                    filename = f"price_point_by_manuf_items_P12M.pkl"
            else:
                    filename = f"price_point_by_manuf_items_P3M.pkl"           
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        print(f"All DataFrames for {hierby} saved to {output_file}.")


# Example Invocation
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels,p12m=True)
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels,p12m=False)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,p12m=True)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,p12m=False)



Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
All DataFrames for SubCategory saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_point_by_brands_items_P12M.pkl.
Query executed successfully for Bj's And Sam

## Price Correlation

In [14]:
def execute_dax_query(entity_name,entity_type, area,market, hierby):
    outputdic = {}
    key =  f"{entity_type} | {entity_name}"

    columns = [
        "Volume Share",'Av Price/KG','Value Share'
    ]
    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
                    DISTINCT(Products[{BrandOrTopB}]),
                    DISTINCT(Calendar[QuarterStart]),
                    DISTINCT(Calendar[End of Week])
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )

    """
    fristcoltot_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    products,
                    Products[{BrandOrTopB}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )

    """
    secondcoltot_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Calendar,
                    Calendar[QuarterStart]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    # Grand total query (not grouped by BrandOrTopB, just Category)
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[Category]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            FILTER('Market', 'Market'[Area] = "{area}"),
            TREATAS({{ "{entity_name}" }}, Market[{market}]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(fristcoltot_query)
            fristttotal_columns = [desc[0] for desc in cursor.description]
            fristtotal_data = cursor.fetchall()       
             
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(secondcoltot_query)
            sectotal_columns = [desc[0] for desc in cursor.description]
            sectotal_data = cursor.fetchall()  
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            maintotal_df = pd.DataFrame(fristtotal_data, columns=fristttotal_columns)
            maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
            
            sectotal_df = pd.DataFrame(sectotal_data, columns=sectotal_columns)
            sectotal_df.columns = sectotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            sectotal_df = sectotal_df.loc[~(sectotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]

            if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            
            if sectotal_df.shape[1] > 1:
                first_col = sectotal_df.columns[0]
                # Ensure datetime
                sectotal_df[first_col] = pd.to_datetime(sectotal_df[first_col], errors='coerce')

                # Format date and append ' total'
                sectotal_df[first_col] = sectotal_df[first_col].dt.strftime('%Y-%m-%d') + " Total"



            if sectotal_df.empty:
                outputdic[key] = sectotal_df
                return outputdic

            if maintotal_df.empty:
                outputdic[key] = maintotal_df
                return outputdic
        
            
            if not maintotal_df.empty:
                df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
                
            df_with_totals['QuarterStart'] = pd.to_datetime(df_with_totals['QuarterStart']).dt.strftime('%Y-%m-%d')

            if not sectotal_df.empty:
                dfsec_with_totals = pd.concat([df_with_totals,sectotal_df], ignore_index=True)

            

            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
          
            if not grand_tot.empty:
                # Set first column value to 'Grand Total', second column to NaN
                grand_tot[df.columns[0]] = 'Grand Total'
                grand_tot[df.columns[1]] = np.nan  # or pd.NA

                # Restrict grand_tot to only columns that appear in df
                grand_tot = grand_tot[df.columns.intersection(grand_tot.columns)]

            # Concatenate all together
            df = pd.concat([dfsec_with_totals, grand_tot], ignore_index=True)

            outputdic[key] = df


            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {area}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {area}: {e}")

    return outputdic



def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for area,market, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{value} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query, entity,value,area,market, hierby)
                            futures[future] = key
       
        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        filename =  f"price_correlation_P3Y.pkl"

        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")

process_dax_queries(entity_hierarchy,hierarchy_levels) 


Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Bj's And Sam's.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.Query executed successfully for NATIONAL.

Query executed successfully for Walmart.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
Query executed successfully for NATIONAL.
Query executed successfully for Bj's And Sam's.
Query executed successfully for Walmart.
All DataFrames saved to c:\Users\AleaaSALAH\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_correlation_P3Y.pkl.


# Read 

In [15]:

# loaded_data = {}
# datasets_path =r"C:\Users\aleaa\Desktop\Slide-Automate Extraction\Pricing\Pricing Datasets NewEX\\"
# datasets = os.listdir(datasets_path)    
# for d in datasets:
#     with open(datasets_path+d, 'rb') as handle:
#         globals()[d.split('.')[0]] = pd.read_pickle(handle)